### 3. Feature Detection and Description 

- Teams will implement feature detection and description using OpenCV. 
- Teams will select and implement at least two feature detection/description 
algorithms (SIFT, SURF, ORB, or others). 
- A comparison of the chosen algorithms' performance is required in the evaluation. 

- Hints for comparison: 
    - Number of detected features: How many keypoints does each algorithm detect in the same image? More keypoints are not always better, but a sufficient number is required for reliable matching. Students could create a plot of the number of features detected vs frame number. 
    - Matching performance: How many correct matches are found by each algorithm? This can be evaluated by visually inspecting a subset of the matches. Students could visualise the matches using a small number of image pairs (e.g., 5-10 pairs). 
    - Computational speed: How long does each algorithm take to detect and describe features? Students can use Python's time module to measure the execution time for this stage. 
    - Robustness to image transformations: How well do the algorithms perform under different conditions, such as changes in scale, rotation, illumination, and viewpoint? This can be done qualitatively by selecting image pairs with significant viewpoint changes and observing the number of matches. 
    - Qualitative assessment: How well do the features visually correspond between frames? Students can create a montage of a few frames, and the feature matches. 
-  Teams must explain the algorithms' parameters and justify their values. 

### SIFT Algorithm

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Load two sample grayscale frames
img1 = cv2.imread('frame1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('frame2.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize SIFT detector
sift = cv2.SIFT_create()  # You can also pass parameters here if tuning

# Time the detection and description process
start_time = time.time()
keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
elapsed_time = time.time() - start_time
print(f"SIFT took {elapsed_time:.4f} seconds to process two images.")

# Visualize keypoints
img_kp1 = cv2.drawKeypoints(img1, keypoints1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(10, 5))
plt.imshow(img_kp1, cmap='gray')
plt.title("SIFT Keypoints in Frame 1")
plt.axis('off')
plt.show()

# Brute-Force matcher with Euclidean distance
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda x: x.distance)

# Draw matches
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:50], None, flags=2)
plt.figure(figsize=(15, 7))
plt.imshow(img_matches)
plt.title("Top 50 SIFT Matches")
plt.axis('off')
plt.show()